In [1]:
import torch
import torchvision
import torch.optim as optim
from torch.autograd import Variable
from image_helper import *
from parse_xml_annotations import *
from features import *
from reinforcement import *
from metrics import *
from collections import namedtuple
import time
import os
import numpy as np
import random


In [11]:
a = torch.tensor([1,2,3,4,5,6])
b = torch.ones(6)
print(torch.sigmoid(a))
print(torch.sigmoid(b))
print(a-b)
print(a*b)

tensor([0.7311, 0.8808, 0.9526, 0.9820, 0.9933, 0.9975])
tensor([0.7311, 0.7311, 0.7311, 0.7311, 0.7311, 0.7311])
tensor([0., 1., 2., 3., 4., 5.])
tensor([1., 2., 3., 4., 5., 6.])


: 

In [2]:
class MultiplyWithSigmoid(nn.Module): #3번째 레이어의 출력과 sigmoid값의 곱
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(25112, 1024)
        self.linear2 = nn.Linear(1024, 1024)
        self.linear3 = nn.Linear(1024, 6)
        self.ReLU1 = nn.ReLU()
        self.Dropout1 = nn.Dropout(0.2)
        self.ReLU2 = nn.ReLU()
        self.Dropout2 = nn.Dropout(0.2)
        self.Sigmoid = nn.Sigmoid()


    def forward(self, x):
        x = self.linear1(x)
        x = self.ReLU1(x)
        x = self.Dropout1(x)
        x = self.linear2(x)
        x = self.ReLU2(x)
        x = self.Dropout2(x)
        linear3_output = self.linear3(x)
        sigmoid_output = self.Sigmoid(linear3_output)
        return linear3_output * sigmoid_output, sigmoid_output
    
def get_multi_q_network(weights_path="0"):
    
    model = MultiplyWithSigmoid()
    if weights_path != "0":
        model.load_state_dict(torch.load(weights_path))
    # init weights by xavier_normal, it may be different with the author's implements
    def weights_init(m):
        if isinstance(m, nn.Linear):
            init.xavier_normal(m.weight.data)
    model.apply(weights_init)
    return model

In [5]:
model = get_multi_q_network()
model = model.cuda()
#2015dqn 적용 model -> current_q_network, target_q_network
agent1_actual = model
agent1_virtual = model

C:\Users\IVCML\AppData\Local\Temp\ipykernel_13968\4050338343.py:33: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight.data)


In [2]:
path_voc = "D:/coding/Multi_object_detection_reinforcement/dataset/VOCdevkit"
# get models 
print("load models")

model_vgg = getVGG_16bn("../models")
model_vgg = model_vgg.cuda()
model = get_q_network()
model = model.cuda()

# define optimizers for each model
optimizer = optim.Adam(model.parameters(),lr=1e-6)
criterion = nn.MSELoss().cuda()   

# get image datas
path_voc_1 = "D:/coding/Multi_object_detection_reinforcement/dataset/VOCdevkit/VOC2007"
path_voc_2 = "D:/coding/Multi_object_detection_reinforcement/dataset/VOCdevkit/VOC2012"
class_object = 'aeroplane' #aeroplane(1) dog(12)
image_names_1, images_1 = load_image_data(path_voc_1, class_object)
image_names_2, images_2 = load_image_data(path_voc_2, class_object)
image_names = image_names_1 + image_names_2
images = images_1 + images_2

print("aeroplane_trainval image:%d" % len(image_names))

# define the Pytorch Tensor
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor
Tensor = FloatTensor

# define the super parameter
multi_agent = 6
epsilon = 1.0
BATCH_SIZE = 100
GAMMA = 0.90
CLASS_OBJECT = 1
steps = 50
epochs = 80
memory = ReplayMemory(1000)


load models


d:\coding\Multi_object_detection_reinforcement\reinforcement.py:84: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight.data)


load imagesD:/coding/Multi_object_detection_reinforcement/dataset/VOCdevkit/VOC2007
total image:238
load imagesD:/coding/Multi_object_detection_reinforcement/dataset/VOCdevkit/VOC2012
total image:670
aeroplane_trainval image:908


In [151]:
x=500
image_name = image_names[x]
print(image_name)
image = images[x]
if x < len(image_names_1):
    annotation = get_bb_of_gt_from_pascal_xml_annotation(image_name, path_voc_1) #xml파일에서 gt box 좌표들 가져온다
else:
    annotation = get_bb_of_gt_from_pascal_xml_annotation(image_name, path_voc_2)   
print(annotation)            
classes_gt_objects = get_ids_objects_from_annotation(annotation)
print(classes_gt_objects)
gt_masks = generate_bounding_box_from_annotation(annotation, image.shape) #gt들을 다 가져온다
print(gt_masks)
original_shape = (image.shape[0], image.shape[1])
print(original_shape)


2009_000801
[[  1.  25. 484. 116. 276.]]
[1.]
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]]
(334, 500)


In [54]:
region_image =[]
size_mask=[]
offset=[] 
history_vector=[] 
region_mask=[] 
state=[]
done=[] 
init_iou=0 #처음에 0으로 설정
# the initial part
for agent in range(multi_agent):
    region_image.append(image) #처음 이미지 자체
    size_mask.append(original_shape) #이미지 shape
    offset.append((0, 0))
    history_vector.append(torch.zeros((4,6)))
    region_mask.append(np.ones((image.shape[0], image.shape[1])))
    state.append(get_state(region_image[agent], history_vector[agent], model_vgg))
    done.append(False)

In [110]:
print(state)

[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]


In [78]:
print(size_mask)
print(region_image[agent])

[(366, 500), (366, 500), (366, 500), (366, 500), (366, 500), (366, 500)]
[[[221 221 221]
  [212 212 212]
  [215 215 215]
  ...
  [226 226 226]
  [221 221 221]
  [223 223 223]]

 [[214 214 214]
  [205 205 205]
  [208 208 208]
  ...
  [222 222 222]
  [217 217 217]
  [219 219 219]]

 [[217 217 217]
  [208 208 208]
  [210 210 210]
  ...
  [223 223 223]
  [218 218 218]
  [220 220 220]]

 ...

 [[ 98  85  69]
  [100  87  71]
  [ 94  81  65]
  ...
  [198 198 198]
  [194 194 194]
  [195 193 194]]

 [[ 93  80  64]
  [ 92  79  63]
  [ 93  80  64]
  ...
  [196 196 196]
  [193 193 193]
  [195 193 194]]

 [[ 86  73  57]
  [101  88  72]
  [103  90  74]
  ...
  [196 196 196]
  [193 193 193]
  [195 193 194]]]


In [104]:
_, _, n = gt_masks.shape 
print(n)
max_iou = 0.0
for k in range(n): #모든 gt 마스크 n개에 대하여 반복
    if classes_gt_objects[k] != CLASS_OBJECT:
        continue
    gt_mask = gt_masks[:,:,k]
    print(np.where(gt_mask))
    print(gt_mask[0])
    iou = calculate_iou(region_mask[agent], gt_mask)
    if max_iou < iou:
        max_iou = iou

3
(array([107, 107, 107, ..., 262, 262, 262], dtype=int64), array([  9,  10,  11, ..., 496, 497, 498], dtype=int64))
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 

In [62]:
print(max_iou)

0.41770491803278686


In [65]:
print(gt_masks)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [99]:
_, _, n = gt_masks.shape 
max_iou = 0.0
min_distance = float('inf')
closest_box = None
for k in range(n): #모든 gt 마스크 n개에 대하여 반복
    if classes_gt_objects[k] != CLASS_OBJECT:
        continue
    gt_mask = gt_masks[:,:,k]
    print(gt_mask[0])
    iou = calculate_iou(region_mask[agent], gt_mask)
    if max_iou < iou:
        max_iou = iou

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [117]:
_, _, n = gt_masks.shape 
min_distance = float('inf')
closest_box = None
for k in range(n): #모든 gt 마스크 n개에 대하여 반복
    if classes_gt_objects[k] != CLASS_OBJECT:
        continue
    print(annotation[k])
    x_diff = ((size_mask[agent][0])/2) - ((annotation[k][2]+annotation[k][1])/2)
    print(x_diff)
    y_diff = ((size_mask[agent][1])/2) - ((annotation[k][4]+annotation[k][3])/2)
    print(y_diff)
    distance = np.sqrt(x_diff**2 + y_diff**2)
    
    if distance < min_distance:
        min_distance = distance
        closest_box = gt_masks[:,:,k]
        closet_gt = annotation[k]
print(closet_gt)        

[  1.   1. 500.  58. 232.]
-67.5
105.0
[  1.   1. 500.  58. 232.]


In [118]:
print(closest_box[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [3]:
def select_action(state):
    if random.random() < epsilon:
        action = np.random.randint(1,7)
    else:
        qval = model(Variable(state)) #epsilon보다 크다면 모델에 넣고 나오는 액션을 택한다 
        _, predicted = torch.max(qval.data,1)
        action = predicted[0] + 1
    return action


Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))
def optimizer_model():
    if len(memory) < BATCH_SIZE: #배치사이즈 보다 메모리가 작으면 리턴 
        return
    transitions = memory.sample(BATCH_SIZE) #메모리에서 샘플을 뽑아옴 
    batch = Transition(*zip(*transitions)) #뽑아온 샘플을 배치로 다시 바꿔준다
    
    non_final_mask = ByteTensor(tuple(map(lambda s: s is not None, batch.next_state)))
    next_states = [s for s in batch.next_state if s is not None]
    non_final_next_states = Variable(torch.cat(next_states), 
                                     volatile=True).type(Tensor)
    state_batch = Variable(torch.cat(batch.state)).type(Tensor)
    action_batch = Variable(torch.LongTensor(batch.action).view(-1,1)).type(LongTensor)
    reward_batch = Variable(torch.FloatTensor(batch.reward).view(-1,1)).type(Tensor)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken
    state_action_values = model(state_batch).gather(1, action_batch)
    
    # Compute V(s_{t+1}) for all next states.
    next_state_values = Variable(torch.zeros(BATCH_SIZE, 1).type(Tensor)) 
    next_state_values[non_final_mask.view(-1,1)] = model(non_final_next_states).max(1)[0]
    
    # Now, we don't want to mess up the loss with a volatile flag, so let's
    # clear it. After this, we'll just end up with a Variable that has
    # requires_grad=False
    next_state_values.volatile = False
    
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    # Compute  loss
    loss = criterion(state_action_values, expected_state_action_values)

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [145]:
offset[agent], region_image[agent], size_mask[agent], region_mask[agent] = get_crop_image_and_mask(original_shape, offset[agent],region_image[agent], size_mask[agent], 5)

In [135]:
print(region_image[agent])

[[[211 211 209]
  [216 216 214]
  [210 210 210]
  ...
  [211 211 211]
  [211 211 211]
  [211 211 211]]

 [[206 206 204]
  [206 206 204]
  [211 211 213]
  ...
  [211 211 211]
  [211 211 211]
  [210 210 210]]

 [[240 240 238]
  [227 227 227]
  [219 219 219]
  ...
  [211 211 211]
  [210 210 210]
  [210 210 210]]

 ...

 [[203 202 200]
  [188 186 187]
  [ 95  93  94]
  ...
  [120 119 127]
  [199 198 206]
  [ 98  97 102]]

 [[240 239 237]
  [171 170 168]
  [144 143 141]
  ...
  [ 71  70  78]
  [147 146 154]
  [177 176 181]]

 [[255 254 252]
  [169 168 166]
  [171 170 168]
  ...
  [183 182 188]
  [ 87  86  92]
  [137 137 139]]]


In [146]:
history_vector[agent] = update_history_vector(history_vector[agent], 5) #이번에 행한 action을 history vector에 추가
next_state = get_state(region_image[agent], history_vector[agent], model_vgg) # 크롭이미지와 history를 이용하여 state 가져옴
                    

In [147]:
print(region_image.size)

AttributeError: 'list' object has no attribute 'size'